# Morphological Inflection in Navajo

## Problem Description

Morphological inflection is a task in computational linguistics wherein the correct form of a word has to be generated from a lemma (base form) and a target morphosyntactic specification, e.g.


```
alzhish  +  V;IND;PFV;NOM(2,SG) -> íínílzhiizh
```


where *alzhish* is the Navajo word for 'dance', and *íínílzhiizh* is the second [2] person , singular [SG], nominative [NOM] form of the verb [V] in the indicative [IND] mood and perfect [PFV] aspect. You can find out more about the annotation schema used for the morphosyntactic specification of the target forms [here](https://unimorph.github.io/doc/unimorph-schema.pdf).

This task finds application in the construction of resources for language acquisition, in statistical machine translation, and in the computational study of language.

The difficulty of the task for a given language depends on the size and complexity of the inflectional paradigm of the language. A large paradigm, which distinguishes between many morphosyntactic properties (number, gender, case, aspect, etc.) and many values for these properties, requires extensive machine learning from sizeable data. While inflectional paragims tend to be highly regular, they can vary in complexity, with many factors guiding which exact morpheme should attach to a given root to mark a specific morphosyntactic property. And irregular forms can often be found too, which cannot be predicted and have to be memorized instead.

Here, we ask you to train a machine learning model to perform morphological inflection in Navajo.

## Your Task

The code below is a near reimplementation of the approach to moprhological inflection presented in [Wu et al.](https://aclanthology.org/2021.eacl-main.163.pdf). The reimplementation uses high-level API from the *transformers* Python library. High-level APIs are convenient as they save us a lot of code-writing, but they also obscure certain aspects of the implementation.

The approach of Wu and colleagues achieves a score of 52.1% for Navajo (as reported [here]()), while the score of the model below lingers about 3 percentage points behind. Study the approach of Wu and colleagues as described in their article to find out what is missing from the model below and make the necessary changes to reproduce their result.   

Notice that due to the stochasticity of the model initialization and training, slight deviations from the expected score are admissible (within 0.5 percentage points).

### Deliverables:
* code
* model accuracy on the test set
* a report of up to 300 words describing the changes made to the model architecture, data processing, training procedure etc.
    * If you do not succeed in reproducing the expected score, comment on what the reason may be.


## Technical Specifications

* All team solutions should be submitted as a modified and compiled copy of this base notebook.
* Do not change cells starting with the `###DO NOT CHANGE THIS CELL###` comment.
* You can use any platform to carry out the development of your model, but for the final submission you have to integrate your solution back into this notebook, adding code to install all dependencies and making sure that your model takes no longer than 8 hours to train.




In [ ]:
! pip install -U accelerate
! pip install -U transformers
! pip install -U evaluate

In [ ]:
###DO NOT CHANGE THIS CELL###
"""
Download the train and dev data
"""

import requests

def download_data(path):
        response = requests.get(path)
        return response.text.strip().split('\n')

train_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.trn'
dev_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.dev'

data = {}
data['train'] = download_data(train_data_path)
data['dev'] = download_data(dev_data_path)

print('Downloaded {} training data samples and {} dev samples.'.format(len(data['train']), len(data['dev'])))
print('Raw data sample:', data['train'][54])

Downloaded 10000 training data samples and 1000 dev samples.
Raw data sample: adiłhash	V;IND;PFV;NOM(3,GRPL)	daʼdiłhash


# Task 1
Modify the code below to achieve a full reimplementation of the approach of Wu and colleagues.

In [ ]:
"""
DATA PREPROCESSING
"""

import re
import regex

def parse_tag(tag):
    tag = re.sub(r"\)|\(|,|;", ' ', tag).split()
    return ''.join(['<{}>'.format(t) for t in tag])

def preprocess_data(raw_data):
        preprocessed_data = []
        for line in raw_data:
          lemma, tag, target = line.split('\t')
          preprocessed_data.append(('{} {}'.format(lemma, parse_tag(tag)),target))
        return preprocessed_data

data['train'] = preprocess_data(data['train'])
data['dev'] = preprocess_data(data['dev'])

print('Preprocessed data sample:', data['train'][54])

chars = set(list(''.join([''.join([d[0].split()[0], d[1]]) for d in data['train']])))
char2id = { char: i for i, char in enumerate(chars)}
tags = list(set(sum([regex.findall(r"<[A-Za-z0-9]*>",d[0]) for d in data['train']], [])))

Preprocessed data sample: ('adiłhash <V><IND><PFV><NOM><3><GRPL>', 'daʼdiłhash')


In [ ]:
"""
TOKENIZER
Input and output word forms are processed one character at a time, while morphosyntactic features are treated as special atomic tokens.
"""

import json
from pathlib import Path
from typing import Optional, Tuple, Dict, List

from transformers import PreTrainedTokenizer
from transformers.tokenization_utils import AddedToken

import warnings

class CustomTokenizer(PreTrainedTokenizer):

    model_input_names = ["input_ids", "attention_mask"]

    def __init__(
        self,
        vocab: Dict[str, int],
        bos_token="<s>",
        eos_token="</s>",
        unk_token="<unk>",
        pad_token="<pad>",
        **kwargs,
    ) -> None:
        # Add extra_ids to the special token list

        self.__token_ids = vocab
        self.__id_tokens: Dict[int, str] = {value: key for key, value in vocab.items()}

        pad_token = AddedToken(pad_token, lstrip=False, rstrip=False) if isinstance(pad_token, str) else pad_token
        bos_token = AddedToken(bos_token, lstrip=False, rstrip=False) if isinstance(bos_token, str) else bos_token
        eos_token = AddedToken(eos_token, lstrip=False, rstrip=False) if isinstance(eos_token, str) else eos_token
        unk_token = AddedToken(unk_token, lstrip=False, rstrip=False) if isinstance(unk_token, str) else unk_token
        self._added_tokens_decoder = {0: pad_token, 1: bos_token, 2: eos_token, 3: unk_token}
        self.offset = len(self._added_tokens_decoder)

        super().__init__(
            bos_token=bos_token,
            eos_token=eos_token,
            unk_token=unk_token,
            pad_token=pad_token,
            **kwargs,
        )

    @property
    def vocab_size(self) -> int:
        return len(self.__token_ids)

    def get_vocab(self) -> Dict[str, int]:
        vocab = {self.convert_ids_to_tokens(i): i for i in range(self.vocab_size + self.offset)}
        vocab.update(self.added_tokens_encoder)
        return vocab

    def _add_eos(self, token_ids: List[int]) -> List[int]:
        return token_ids + [self.eos_token_id]

    def create_token_type_ids_from_sequences(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        eos = [self.eos_token_id]

        if token_ids_1 is None:
            return len(token_ids_0 + eos) * [0]
        return len(token_ids_0 + eos + token_ids_1 + eos) * [0]

    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        token_ids_0 = self._add_eos(token_ids_0)
        if token_ids_1 is None:
            return token_ids_0
        else:
            token_ids_1 = self._add_eos(token_ids_1)
            return token_ids_0 + token_ids_1

    def _tokenize(self, text: str, **kwargs):
        return list(text)

    def _convert_token_to_id(self, token: str) -> int:
        return self.__token_ids[token]+self.offset if token in self.__token_ids else self.unk_token_id

    def _convert_id_to_token(self, index: int) -> str:
        return self.__id_tokens[index-self.offset] if index-self.offset in self.__id_tokens else self.unk_token

    def convert_tokens_to_string(self, tokens):
        return "".join(tokens)

    def save_vocabulary(self, save_directory: str,
                        filename_prefix: Optional[str] = None) -> Tuple[str]:
        if filename_prefix is None:
            filename_prefix = ''
        vocab_path = Path(save_directory, filename_prefix + 'vocab.json')
        json.dump(self.__token_ids, open(vocab_path, 'w'))
        return str(vocab_path),

tokenizer = CustomTokenizer(char2id, additional_special_tokens=tags, max_len=100)
print('Tokenization example:', tokenizer.tokenize(data['train'][54][0]))

Tokenization example: ['a', 'd', 'i', 'ł', 'h', 'a', 's', 'h', ' ', '<V>', '<IND>', '<PFV>', '<NOM>', '<3>', '<GRPL>']


In [ ]:
"""
MODEL
The model is based on the t5 model architecture: a transformer-based model with an encoder
and a decoder, trained to take an input sequence (the lemma and the target tag)
and to generate an output sequence (the target form).
"""

from transformers import T5Config, T5ForConditionalGeneration

config = T5Config(d_ff=1024,
              d_model=256,
              num_layers=4,
              num_decoder_layers=4,
              num_heads=4,
              dropout_rate=0.2,
              vocab_size=len(tokenizer))
model = T5ForConditionalGeneration(config)
model.config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.decoder_start_token_id = tokenizer.bos_token_id
model.generation_config.max_new_tokens = 32
model.generation_config.eos_token_id = tokenizer.eos_token_id


In [ ]:
"""
UTILITIES
"""

from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.data = data
        self.tokenizer = tokenizer
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        input, target = self.data[idx]
        return {"input_ids": self.tokenizer(input, padding='longest',
                                            truncation=True,
                                            add_special_tokens=True)['input_ids'],
                "labels": self.tokenizer(target, padding='longest',
                                         truncation=True,
                                         add_special_tokens=True)['input_ids']}

def postprocess_data(token_ids):
    token_ids = np.where(token_ids != -100, token_ids, tokenizer.pad_token_id)
    return tokenizer.batch_decode(token_ids, skip_special_tokens=True)

In [ ]:
###DO NOT CHANGE THIS CELL###
"""
MAIN METRIC
"""

import evaluate
import numpy as np
import random

def compute_metrics(eval_preds):
    metric = evaluate.load("exact_match")
    preds, labels = eval_preds
    decoded_preds = postprocess_data(preds)
    decoded_labels = postprocess_data(labels)

    # During development, you can uncomment the lines to see what predictions your model makes
    # ks = random.choices(list(range(len(decoded_preds))), k=15)
    # print('Predicted:', [decoded_preds[k] for k in ks])
    # print('Targets:', [decoded_labels[k] for k in ks])
    # print('___________________________________________________________________')

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return result

In [ ]:
"""
TRAINING
"""

from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

dataset = {'train': CustomDataset(data['train'], tokenizer),
           'dev': CustomDataset(data['dev'], tokenizer)}
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

training_args = Seq2SeqTrainingArguments(
    max_steps=20000,
    per_device_train_batch_size=400,
    learning_rate = 0.001,
    lr_scheduler_type='inverse_sqrt',
    warmup_steps=4000,
    adam_beta2=0.98,
    label_smoothing_factor=0.1,
    evaluation_strategy="steps",
    eval_steps=400,
    eval_delay=400,
    save_strategy="steps",
    save_steps=400,
    predict_with_generate=True,
    metric_for_best_model='exact_match',
    save_total_limit=1,
    logging_strategy="steps",
    logging_steps=400,
    output_dir='baseline_0.2',
    overwrite_output_dir=True,
    load_best_model_at_end=True
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset['train'],
    eval_dataset=dataset['dev'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Exact Match
400,2.768400,1.623577,0.032000
800,1.512300,1.423992,0.137000
1200,1.178200,1.388152,0.160000
1600,1.015200,1.436030,0.209000
2000,0.925100,1.421848,0.221000
2400,0.868800,1.440768,0.238000
2800,0.835900,1.462445,0.279000
3200,0.815400,1.440195,0.296000
3600,0.802400,1.423015,0.321000
4000,0.793900,1.407912,0.363000


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=20000, training_loss=0.8132031356811523, metrics={'train_runtime': 3760.6997, 'train_samples_per_second': 2127.264, 'train_steps_per_second': 5.318, 'total_flos': 7613745476812800.0, 'train_loss': 0.8132031356811523, 'epoch': 800.0})

In [ ]:
###DO NOT CHANGE THIS CELL###
"""
EVALUATION
"""

test_data_path = 'https://raw.githubusercontent.com/sigmorphon/2023InflectionST/main/part1/data/nav.tst'
test_data = download_data(test_data_path)
test_dataset = CustomDataset(preprocess_data(test_data), tokenizer)
result = trainer.evaluate(test_dataset)
test_accuracy_wu = result['eval_exact_match']



### Test Accuracy

In [ ]:
###DO NOT CHANGE THIS CELL###
print('Test accuracy:', test_accuracy_wu)

Test accuracy: 0.488


### Report
[Describe your approach in up to 300 words here.]

# Task 2
Build a better morphological inflector.

In [ ]:
# your code goes here - make sure the final test accuracy is saved as test_accuracy

### Test Accuracy

In [ ]:
###DO NOT CHANGE THIS CELL###
print('Test accuracy:', test_accuracy)

### Report
[Describe your approach in up to 300 words here.]